In [ ]:
import os
import json
import numpy as np
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Input, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras import backend as K

# Function to apply Gaussian pyramid to an image
def gaussian_pyramid(image, levels=3):
    pyramid = [image]
    for _ in range(levels - 1):
        image = cv2.GaussianBlur(image, (5, 5), 0)
        image = cv2.resize(image, (int(image.shape[1] / 2), int(image.shape[0] / 2)))
        pyramid.append(image)
    return pyramid

# Set the path to your dataset
dir0 = '../d/input/indian-food-images-dataset/'

# Collect paths and labels
paths = []
labels = []
for dirname, _, filenames in os.walk(dir0):
    for filename in filenames:
        if filename[-4:] == '.jpg':
            paths += [os.path.join(dirname, filename)]
            labels += [dirname.split('/')[-1]]

# Create a DataFrame for easier handling
import pandas as pd
data = pd.DataFrame({'Path': paths, 'Label': labels})

# Split the dataset into training and validation sets
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

# Define constants
input_size = (224, 224)
batch_size = 32
epochs = 2
num_classes = len(data['Label'].unique())
pyramid_levels = 3  # Number of levels in the Gaussian pyramid

# Save data generator configuration and class information
data_info = {
    'data_generator_config': {
        'rescale': 1./255,
        'shear_range': 0.2,
        'zoom_range': 0.2,
        'horizontal_flip': True
    },
    'class_labels': list(data['Label'].unique())
}

with open('../d/working/data_info.json', 'w') as json_file:
    json.dump(data_info, json_file)

# Load data generator configuration and class information
with open('../d/working/data_info.json', 'r') as json_file:
    loaded_data_info = json.load(json_file)
    loaded_data_generator_config = loaded_data_info['data_generator_config']
    loaded_class_labels = loaded_data_info['class_labels']

# Build ResNet50 model without top layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers to handle Gaussian pyramid
input_layer = Input(shape=(None, None, 3))
gaussian_pyramid_layer = Lambda(lambda x: K.map_fn(lambda y: gaussian_pyramid(y[0]), x, dtype='float32'))(input_layer)

model = Sequential()
model.add(gaussian_pyramid_layer)
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Data augmentation
train_datagen = ImageDataGenerator(**loaded_data_generator_config)

val_datagen = ImageDataGenerator(rescale=1./255)

# Load and preprocess the training dataset
train_generator = train_datagen.flow_from_dataframe(dataframe=train_data,
                                                    x_col='Path',
                                                    y_col='Label',
                                                    target_size=input_size,
                                                    batch_size=batch_size,
                                                    class_mode='categorical')

# Load and preprocess the validation dataset
val_generator = val_datagen.flow_from_dataframe(dataframe=val_data,
                                                x_col='Path',
                                                y_col='Label',
                                                target_size=input_size,
                                                batch_size=batch_size,
                                                class_mode='categorical')

# Train the model
history = model.fit(train_generator,
                    epochs=epochs,
                    validation_data=val_generator)

# Save the trained model
model.save('../d/working/food_recognition_model_with_pyramid.h5')

# Evaluate the model
test_loss, test_accuracy = model.evaluate(val_generator)
print(f'Test loss: {test_loss}, Test accuracy: {test_accuracy}')
